In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from google.colab import files
from IPython.display import HTML, display

import numpy as np
import io
import re
from copy import deepcopy 

# REQUIRED
testFileName = 'test_4.txt'
trainFileName = 'train_4.txt'
classAttributeIndex = 14
attributesIgnore = [2, 4, 10, 11]

# PARAMETERS
dataSplitRatio = 0


In [0]:
# Function to read a file
def readFile( fileName ):
  with open( fileName, 'r' ) as f:
    lines = f.read().split( '\n' )
    return lines

print("#### FILE DATA ####")
trainData = readFile( trainFileName )
testData = readFile( testFileName )
for line in testData:
  print( line )

In [0]:
# Converting the file data into a 2D array
def tabulateData( data, delimiter = ' ', hasHeader = True ):
  X = []
  for line in data:
    words = line.split(delimiter)
    X.append(words)
  return X

print("#### TABULATED DATA ####")
trainTabulatedData = tabulateData( trainData )
testTabulatedData = tabulateData( testData )
display(HTML(
   '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in testTabulatedData)
       )
))


In [0]:
# Removing data points which consists of null values
def preprocessData( tabulatedData, classAttributeIndex, train = True ):
  X = []
  Y_train = [ ]
  requiredLength = len( tabulatedData[0] )
  for dataPoint in tabulatedData:
    if( len(dataPoint) < requiredLength ):
      continue
    # if "none" in dataPoint:
    #   continue
    X.append( dataPoint[ :requiredLength ] )
  
  X = np.asanyarray(X)
  if(train is True):
    Y_train = X[:, classAttributeIndex]
    X = np.delete(X, classAttributeIndex, axis = 1)
  return X, Y_train

print("#### PREPROCESSED DATA ####")
X_train, Y_train = preprocessData( trainTabulatedData, classAttributeIndex = classAttributeIndex )
X_test, Y_test = preprocessData( testTabulatedData, classAttributeIndex = classAttributeIndex, train = False )
print(X_train[10,:])
# print(Y_train)


In [0]:
def categorical_distance(ptA, ptB):
  diff = ( ptA == ptB )
  return np.size(diff) - np.sum(diff)

def euclidean_distance(ptA, ptB):
  a = ptA.astype(np.float)
  b = ptB.astype(np.float)
  return (np.sum((a - b)**2)**0.5)

def distance(ptA, ptB, numeric_attributes, categorical_attributes):
  dist = 0
  dist += euclidean_distance(ptA[numeric_attributes], ptB[numeric_attributes])
  dist += categorical_distance(ptA[categorical_attributes], ptB[categorical_attributes])
  return dist

In [0]:
def findAttributeTypes(X):
  N, M = np.shape(X)
  i = 0
  dataSet = X[0,:]
  while('?' in dataSet):
    i += 1
    dataSet = X[i, :]
  categorical_attributes = []
  numeric_attributes = []
  array = dataSet

  for i in range(len(array)):
    regex_output = None
    x = re.search('^[A-Za-z]+[-]*', array[i])
    if x is not None:
      categorical_attributes.append(i)
      continue
    x = re.search('^[0-9]+[.]*[0-9]+$', array[i])
    if x is not None:
      numeric_attributes.append(i)
      continue
    else:
      categorical_attributes.append(i)

  return numeric_attributes, categorical_attributes


# Function to process Data that is removing the columns
def processData( data, removeColumns ):
  data = np.delete( data, removeColumns, axis = 1 )
  numeric_attributes, categorical_attributes = findAttributeTypes( data )
  return data, numeric_attributes, categorical_attributes

X_train, numeric_attributes, categorical_attributes = processData( X_train, attributesIgnore )
X_test, numeric_attributes2, categorical_attributes2 = processData( X_test, attributesIgnore )

print(numeric_attributes)
print(categorical_attributes)
print(X_train[0,:])

In [0]:
def getMissedDataPoint( data ):
  N, M = np.shape(data)
  for i in range(N): 
    point = data[i, :]
    if '?' in point:
      return i
  return -1

def getKNNeighbours(X_train, Y_train, testPoint, k, numeric_attributes, categorical_attributes):
    dist = np.empty((1,3))

    # Finding distance with all the training nodes and storing in dist matrix 
    N_train, M = np.shape(X_train)

    for j in range( N_train ):
      trainPoint = X_train[j, :]
      if not '?' in trainPoint:
        temp = np.array([[ j, distance( trainPoint, testPoint, numeric_attributes, categorical_attributes ), Y_train[j] ]])
        dist = np.append( dist, temp, axis = 0 )
    dist = np.delete(dist, 0, axis = 0)

    # Sorting the distances
    dist = dist[dist[:, 1].argsort()] 

    # Selecting top K elements as our neighbours
    neighbours = dist[:k, :]

    return neighbours

def fillTrainSet( data, Y, numeric_attributes, categorical_attributes ):
  N, M = np.shape( data )
  
  # i = getMissedDataPoint( data )
  for i in range(10):
    myPoint = data[i, :]
    missingAttributesIndex = []
    print("-------------------------------")
    print("Data point with missing value:")
    print(data[i, :])
    for j in range( len(myPoint) ):
      if '?' in myPoint[j]:
        missingAttributesIndex.append(j)
    copy_myPoint = deepcopy(myPoint)
    copy_data = deepcopy(data)

    copy_myPoint = np.delete(copy_myPoint, missingAttributesIndex )
    copy_data = np.delete(copy_data, missingAttributesIndex, axis = 1)

    numeric_attributes, categorical_attributes = findAttributeTypes( copy_data )
    
    myNeighbour = getKNNeighbours( copy_data[10:,:], Y, copy_myPoint, 1, numeric_attributes, categorical_attributes )

    for j in range( len(myPoint) ):
      if '?' in myPoint[j]:
        # Adding 10 as we started with 10 training instances
        data[i, j] = data[ int(myNeighbour[0][0]) + 10, j ]
    print("My nearest equivalent:")
    print((data[int(myNeighbour[0][0])+10, :]) ) 
    print("My updated value:")
    print(data[i, :])
  return data
    

X_train = fillTrainSet(X_train, Y_train, numeric_attributes, categorical_attributes )



In [0]:
numeric_attributes, categorical_attributes = findAttributeTypes(X_train[10:, :])

def predict( neighbours ):
  # Finding count class
  countClass = dict()
  for neighbourClass in neighbours[:, 2]:
    if( neighbourClass in countClass ):
      countClass[neighbourClass] += 1
    else:
      countClass[neighbourClass] = 1

  # Finding prediction
  maxCount = -1
  predClass = None
  for countClassKey in countClass:
    if( maxCount < countClass[ countClassKey ] ):
      maxCount = countClass[ countClassKey ]
      predClass = countClassKey

  return predClass

N_test, M = np.shape(X_test)
for i in range(N_test):
  testPoint = X_test[i]
  numeric_attributes, categorical_attributes = findAttributeTypes(X_train)
  neighbours = getKNNeighbours(X_train, Y_train, testPoint, 3 , numeric_attributes, categorical_attributes)
  print(testPoint)
  print("Predicted Class: " + str(predict(neighbours)))
  print("-------------")
